# About this lab


Machine Learning and artificial intelligence are becoming prevalent in the software industry. While there is Pre-built AI & conversational AI,  we'd still run into customer scenarios when custom model / code development is required. And it is essential  to get insights into what it's like To bring these scenarios to life by extending current work Or building from scratch 
One of the common areas of artificial intelligence is Computer vision and in it, specifically object detection. 
The lab is designed to help you understand at a very high level The constructs and steps involved in an object detection scenario. Many models and techniques exist in the market we will be using tensorflow  and faster-RCNN model for this lab. 


# What is Transfer Learning? 

Transfer Learning is a branch of Machine Learning which builds custom models on top of existing models. Example, RESNET models  trained on ImageNet & COCO datasets, can be extended to identify objects outside of the above mentioned datasets. Advantages :
1. Less Data
2. Use all the goodness of parent or base model.
3. Training new models is faster


# The lab scenario 


In this lab, we will develop a model to identify & segment the app icons in given input image/video. 
We will start by creating & prr-processing input data for the experiment, followed by configuring training, running modell training & exporting the trained model for inference. 


# What is Object Detection?


Object Detection is a type of artificial intelligence task which tries to identify (predict) the object in given input. If you have worked with imgage classification scenarios, then learning object detection should be easier for you. Best way is to start with concepts related to Convolution, Deep Learning and other Digital processing if you want to dig deeper

# The Approach 

While we have done a lot of work for you have a seamless lab & learning experience, we have primarily built this lab of top of tensorflow object detection steps. The lab environments already has he below steps performed :
1. Downloading the tensorflow github repo for object detection
2. Generating supporting files (protocol buffers)
3. Ensuring other supporting packages related to image processing are installed.
4. Downloading pre-ran / pre-built models so that you can complete the lab in allocated time.


In [ ]:
!echo $PYTHONPATH

In [ ]:
!python3 ~/ready-labs/tf-files/research/object_detection/builders/model_builder_test.py

# The pre-processing

It is important to note that all input files have same resolution (size) for a more accurate model


 - We will use VOTT tool to first annotate objects and specify location by drawing bounding box around the object of interest (in this case, app icons)
 - For the session, we will be working with only 6 app icons.
 - We will resize all input images to 500x500 
 - Resize code available here

- Download the VOTT Tool from [here](https://github.com/Microsoft/VoTT/releases/download/1.5.0/VoTT-win32-x64.zip) 
- We will use the 'PASCAL VOC' format to export the tags

# The Code

In [ ]:
#steps to be performed for models
# copy data2 to custom-training
#create model_files directory in custom-training & sub-directory called 
#copy from storage (Ashish) contents of data2 folder
#create a train folder

Image tagging/preprocessing=.. talk about formats, like Pascal VOC, etc.


# Download models (ignore for now)

In [ ]:
!pwd

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

os.chdir(rootdir)  


rootdir = '/home/labadmin/ready-labs/tf-files/research/object_detection/custom-training/'
labels_dir = 'data2/'
model_extract_dir = 'model_test'  # insert folder name here

os.mkdir(model_extract_dir)

MODEL_NAME = 'faster_rcnn_resnet101_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(labels_dir, 'mscoco_label_map.pbtxt')
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, model_extract_dir + MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
tar_file.extractall(path=model_extract_dir)

In [ ]:
#copy the pipeline config file to the custom directory

In [ ]:
import os
os.getcwd()

# Pre-processing : creating inputs

In [ ]:
#%cd /home/labadmin/ready-labs/tf-files/research

In [ ]:
#%mkdir object_detection/custom-training/models/train

In [ ]:
#python /dataset_tools/create_pascal_tf_record.py --output_path=object_detection/custom-training/data2/train/pascal.record --data_dir=object_detection/custom-training/data2/train --set=train --label_map_path=object_detection/custom-training/data2/train/pascal_label_map.pbtxt

<b> Pay attention to comments in the code below. Inputs needed</b>

In [ ]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

r"""Convert raw PASCAL dataset to TFRecord for object_detection.

Example usage:
    python object_detection/dataset_tools/create_pascal_tf_record.py \
        --data_dir=/home/user/VOCdevkit \
        --year=VOC2012 \
        --output_path=/home/user/pascal.record
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import io
import logging
import os

from lxml import etree
import PIL.Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util


input_data_dir = rootdir + 'data2/test' #toggle here between train & test
input_set_flag ='val' #toggle for tarin or val[idation]
input_annotations_dir = 'Annotations' 
output_path = rootdir + 'data2/test/pascal.record' #toggle for train  or val folders
label_map_path = rootdir + 'data2/test/pascal_label_map.pbtxt' #toggle for train  or val folders
ignore_difficult_instances = False

flags = tf.app.flags

SETS = ['train', 'val', 'trainval', 'test']
classes_texts = ['camera','phone','calendar','mail','message']

if input_set_flag not in SETS:
    raise ValueError('set must be in : {}'.format(SETS))
    
data_dir = input_data_dir #FLAGS.data_dir

writer = tf.python_io.TFRecordWriter(output_path) #FLAGS.output_path

label_map_dict = label_map_util.get_label_map_dict(label_map_path) #FLAGS.label_map_path

for class_ in classes_texts:
    examples_path = os.path.join(data_dir, 'ImageSets', 'Main',
                                 class_ + '_'+ input_set_flag +'.txt')
    annotations_dir = os.path.join(data_dir, input_annotations_dir)
    examples_list = dataset_util.read_examples_list(examples_path)
    
    for idx, example in enumerate(examples_list):
      if idx % 100 == 0:
        logging.info('On image %d of %d', idx, len(examples_list))
      if(example != ''):
        path = os.path.join(annotations_dir, example + '.xml')
        with tf.gfile.GFile(path, 'r') as fid:
          xml_str = fid.read()
        xml = etree.fromstring(xml_str)
        data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']

        tf_example = dict_to_tf_example(data, input_data_dir, label_map_dict,
                                      ignore_difficult_instances)
        writer.write(tf_example.SerializeToString())

writer.close()

def dict_to_tf_example(data,
                       dataset_directory,
                       label_map_dict,
                       ignore_difficult_instances=False,
                       image_subdirectory='JPEGImages'):
  """Convert XML derived dict to tf.Example proto.

  Notice that this function normalizes the bounding box coordinates provided
  by the raw data.

  Args:
    data: dict holding PASCAL XML fields for a single image (obtained by
      running dataset_util.recursive_parse_xml_to_dict)
    dataset_directory: Path to root directory holding PASCAL dataset
    label_map_dict: A map from string label names to integers ids.
    ignore_difficult_instances: Whether to skip difficult instances in the
      dataset  (default: False).
    image_subdirectory: String specifying subdirectory within the
      PASCAL dataset directory holding the actual image data.

  Returns:
    example: The converted tf.Example.

  Raises:
    ValueError: if the image pointed to by data['filename'] is not a valid JPEG
  """
  
  img_path = os.path.join(image_subdirectory, data['filename'])
  full_path = os.path.join(dataset_directory, img_path)
  full_path = full_path + '.' + data['path'].rsplit('.',2)[1] #extension #file_extension 
  with tf.gfile.GFile(full_path, 'rb') as fid:
    encoded_jpg = fid.read()
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = PIL.Image.open(encoded_jpg_io)
  if image.format != 'JPEG':
    raise ValueError('Image format not JPEG')
  key = hashlib.sha256(encoded_jpg).hexdigest()

  width = int(data['size']['width'])
  height = int(data['size']['height'])

  xmin = []
  ymin = []
  xmax = []
  ymax = []
  classes = []
  classes_text  = []
  
  truncated = []
  poses = []
  difficult_obj = []
  if 'object' in data:
    for obj in data['object']:
      difficult = bool(int(obj['difficult']))
      if ignore_difficult_instances and difficult:
        continue

      difficult_obj.append(int(difficult))

      xmin.append(float(obj['bndbox']['xmin']) / width)
      ymin.append(float(obj['bndbox']['ymin']) / height)
      xmax.append(float(obj['bndbox']['xmax']) / width)
      ymax.append(float(obj['bndbox']['ymax']) / height)
      classes_text.append(obj['name'].encode('utf8'))
      classes.append(label_map_dict[obj['name']])
      truncated.append(int(obj['truncated']))
      poses.append(obj['pose'].encode('utf8'))

  example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(data['filename'].encode('utf8')),
      'image/source_id': dataset_util.bytes_feature(data['filename'].encode('utf8')),
      'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
      'image/object/difficult': dataset_util.int64_list_feature(difficult_obj),
      'image/object/truncated': dataset_util.int64_list_feature(truncated),
      'image/object/view': dataset_util.bytes_list_feature(poses),
  }))
  return example



  

# Creating Validation sets

Run the above commands with set=val & updated file paths

In [ ]:
#!python3 object_detection/dataset_tools/create_pascal_tf_record.py --output_path=object_detection/custom-training/data2/test/pascal.record --data_dir=object_detection/custom-training/data2/test --set=val --label_map_path=object_detection/custom-training/data2/test/pascal_label_map.pbtxt

# Training with RESNET101

Locate and modify the pipeline.config or the faster_rcnn_resnet101_coco.config as :

- update config file
- num classes : 6
- input_path : line 30
- label_map_path :  line 132
- eval_config : max evals = 3
- train_input_reader : input_path line 116
- label_map_path : line 118
- fine_tune_checkpoint : line 106

In [ ]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Binary to run train and evaluation on object detection model."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import flags

import tensorflow as tf

from object_detection import model_hparams
from object_detection import model_lib

modeldir = rootdir + 'model_files'
train_dir = rootdir + modeldir + 'train'
pipeline_config_path_custom = rootdir + 'faster_rcnn_resnet101_coco.config'

#update_model_dir
#flags.DEFINE_string(
#    'model_dir', None, 'Path to output model directory '
#'where event and checkpoint files will be written.')

#update pipeline config
#flags.DEFINE_string('pipeline_config_path', pipeline_config_path, 'Path to pipeline config '
#                    'file.')

#not mandatory  : start

FLAGS = flags.FLAGS
#not mandatory end

#flags.mark_flag_as_required('model_dir')
#flags.mark_flag_as_required('pipeline_config_path')
config = tf.estimator.RunConfig(model_dir=modeldir)

train_and_eval_dict = model_lib.create_estimator_and_inputs(
      run_config=config,
      hparams=model_hparams.create_hparams(FLAGS.hparams_overrides),
      pipeline_config_path= pipeline_config_path_custom, #FLAGS.pipeline_config_path,
      train_steps=FLAGS.num_train_steps,
      sample_1_of_n_eval_examples=FLAGS.sample_1_of_n_eval_examples,
      sample_1_of_n_eval_on_train_examples=(
          FLAGS.sample_1_of_n_eval_on_train_examples))
estimator = train_and_eval_dict['estimator']
train_input_fn = train_and_eval_dict['train_input_fn']
eval_input_fns = train_and_eval_dict['eval_input_fns']
eval_on_train_input_fn = train_and_eval_dict['eval_on_train_input_fn']
predict_input_fn = train_and_eval_dict['predict_input_fn']
train_steps = train_and_eval_dict['train_steps']

if FLAGS.checkpoint_dir:
    if FLAGS.eval_training_data:
      name = 'training_data'
      input_fn = eval_on_train_input_fn
    else:
      name = 'validation_data'
      # The first eval input will be evaluated.
      input_fn = eval_input_fns[0]
    if FLAGS.run_once:
      estimator.evaluate(input_fn,
                         num_eval_steps=None,
                         checkpoint_path=tf.train.latest_checkpoint(
                             FLAGS.checkpoint_dir))
    else:
      model_lib.continuous_eval(estimator, FLAGS.checkpoint_dir, input_fn,
                                train_steps, name)
else:
    train_spec, eval_specs = model_lib.create_train_and_eval_specs(
        train_input_fn,
        eval_input_fns,
        eval_on_train_input_fn,
        predict_input_fn,
        train_steps,
        eval_on_train_data=False)

    # Currently only a single Eval Spec is allowed.
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_specs[0])

In [ ]:
#Export & host in ACI

In [ ]:
!pwd

#file actaull inside /.../// in tensorflow spurce. copied here just for demo/lab
from object_detection.utils import dataset_util

# Export resnet101 with more data

In [ ]:
#see the value of the max ckpt number
#run the script below
#copy  file export_inference_graph.py from ~/ready-labs/tf-files/research/object_detection/ to custom-training folder

In [ ]:
!python3 object_detection/custom-training/export_inference_graph.py --input_type=image_tensor --pipeline_config_path=object_detection/custom-training/faster_rcnn_resnet101_coco.config --trained_checkpoint_prefix=object_detection/custom-training/model_final/model.ckpt-296 --output_directory=object_detection/custom-training/fine_tuned_model

<b> Ensure to upload test images to the object_detection/test_images folder

# Run the object detection notebook [Object detection notebook](tf-files/research/object_detection/object_detection_tutorial.ipynb)




Comment the Download model code and the following changes :



- MODEL_NAME = 'custom-training/fine_tuned_model/
- PATH_TO_LABELS = os.path.join('custom-training/data2/train', 'pascal_label_map.pbtxt')

# Deployment

Python SDK. ACI & logic apps